In [ ]:
# 데이터 js로 변환

import pandas as pd
import json

# --- 장르 리스트 변환 ---
def tidy_genres(gstr):
    if pd.isnull(gstr):
        return []
    return [g.strip() for g in str(gstr).split(',') if g.strip()]

# --- 문자열을 불린으로 변환 ---
def boolify(val):
    sval = str(val).lower().strip()
    return sval in ['true', '1', 'yes', 'y']

# --- 행(row)을 웹툰 객체로 변환 ---
def to_webtoon_obj(row):
    return {
        'id': int(row['id']) if not pd.isnull(row['id']) else 0,
        'title': str(row['title']),
        'author': str(row['author']),
        'genre': tidy_genres(row['genre']),
        'description': str(row['description']) if not pd.isnull(row['description']) else '',
        'rating': float(row['rating']) if not pd.isnull(row['rating']) else 0,
        'completed': boolify(row['completed']),
        'age': str(row['age']) if not pd.isnull(row['age']) else '',
        'free': boolify(row['free']),
        'link': str(row['link']) if not pd.isnull(row['link']) else '#',
        'img': str(row['thumbnail_url']) if not pd.isnull(row['thumbnail_url']) else ''
    }

# --- CSV 읽기 (UTF-8 확정) ---
csv_path = './data/naver.csv'
df = pd.read_csv(csv_path, encoding='utf-8')

# --- 각 행을 JS용 객체로 변환 ---
webtoons = [to_webtoon_obj(row) for _, row in df.iterrows()]

# --- 장르 / 연령 추출 ---
all_genres = sorted({g for w in webtoons for g in w['genre']})
all_ages = sorted({w['age'] for w in webtoons if w['age']})

# --- JS 파일로 저장 ---
data = {'webtoons': webtoons, 'genres': all_genres, 'ages': all_ages}
with open('webtoons_data.js', 'w', encoding='utf-8') as f:
    f.write('const webtoonsData = ')
    json.dump(data, f, ensure_ascii=False, indent=2)
    f.write(';')

print(" webtoons_data.js 파일이 성공적으로 생성되었습니다. (UTF-8 인코딩 사용)")


✅ webtoons_data.js 파일이 성공적으로 생성되었습니다. (UTF-8 인코딩 사용)


In [ ]:
# 썸네일 크롤링

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import time
from tqdm import tqdm  # 진행 상황을 시각적으로 보여주기 위한 라이브러리

# --- 1. Selenium 및 웹 드라이버 설정 ---
def setup_driver():
    """Selenium 웹 드라이버를 설정하고 반환합니다."""
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # 브라우저 창 없이 백그라운드 실행
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36")

    service = Service()
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

# --- 2. naver.csv 파일 로드 ---
try:
    df = pd.read_csv('naver.csv')
except FileNotFoundError:
    print("'naver.csv' 파일을 찾을 수 없습니다. 파일이 코드와 동일한 디렉토리에 있는지 확인하세요.")
    exit()

# 크롤링 결과를 저장할 리스트
results = []

# --- 3. 웹 드라이버 실행 ---
driver = setup_driver()
print("웹 드라이버 실행 완료. 크롤링을 시작합니다...")

# --- 4. 모든 타이틀을 순회하며 크롤링 ---
# tqdm을 사용하여 진행률 표시
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="웹툰 크롤링 중"):
    title = row['title']
    webtoon_link = row['link']
    
    thumbnail_url = None  # 기본값 설정
    try:
        # 웹툰 페이지로 이동
        driver.get(webtoon_link)
        
        # 페이지 로드를 위한 짧은 대기 (서버 부하 감소 목적)
        time.sleep(1) # 1초 대기

        # 지정된 class 이름으로 img 요소 찾기
        img_element = driver.find_element(By.CSS_SELECTOR, "img.Poster__image--d9XTI")
        thumbnail_url = img_element.get_attribute('src')

    except NoSuchElementException:
        # 이미지를 찾지 못한 경우
        print(f"\n[오류] '{title}' 웹툰에서 썸네일 이미지를 찾을 수 없습니다.")
        thumbnail_url = "Not Found"
    except Exception as e:
        # 그 외 예외 처리
        print(f"\n[오류] '{title}' 크롤링 중 에러 발생: {e}")
        thumbnail_url = "Error"
    
    # 결과 저장
    results.append({
        'title': title,
        'thumbnail_url': thumbnail_url
    })

# --- 5. 드라이버 종료 ---
driver.quit()
print("크롤링 완료. 드라이버를 종료합니다.")

# --- 6. 결과를 새로운 DataFrame으로 만들고 CSV 파일로 저장 ---
result_df = pd.DataFrame(results)

# 원본 데이터와 병합 (선택 사항)
# final_df = pd.merge(df, result_df, on='title', how='left')

# 결과 확인
print("\n--- 크롤링 결과 (상위 5개) ---")
print(result_df.head())

# CSV 파일로 저장
output_filename = 'naver_webtoon_thumbnails.csv'
result_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"\n모든 결과가 '{output_filename}' 파일로 저장되었습니다.")

